# Timelink developper tutorial
## Using ORM for database access

`timelink` uses SQLAlchemy ORM models for database access.

The `timelink` database schema is based on the concept of hierarchical classes.

At the root of the hierarchy is the `Entity` class. Basic builtin classes specialize `Entity`: `Person`, `Object`, `Source`, `Act`.

The hierarchy is managed through SQLAlchemy "Joined table inheritance".

For documentation see: https://docs.sqlalchemy.org/en/20/orm/inheritance.html

### Initialize a notebook object

You need docker to run this tutorial

In [1]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook(db_name='orm_tutorial')
tlnb.db.drop_db()
tlnb.db.create_db()
tlnb.print_info()

Timelink version: 1.1.14
Project name: tutorial
Project home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial
Database type: sqlite
Database name: orm_tutorial
Kleio image: timelinkserver/kleio-server
Kleio server token: SuoUN...
Kleio server URL: http://127.0.0.1:8089
Kleio server home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial
Kleio server container: dazzling_jepsen
Kleio version requested: latest
Kleio server version: 12.6.573 (2024-07-29 08:30:28)
SQLite directory: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial/database/sqlite
Call print_info(show_token=True) to show the Kleio Server token
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=tutorial, project_home=/Users/jrc/develop/timelink-py/tests/timelink-home/projects/tutorial, db_type=sqlite, db_name=orm_tutorial, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=postgres, postgres_versi

#### Get the database object from the Notebook object

This will make it easier to access the database functions, but you can use the `tlnb.db` expression as well.

Show the tables in the database

In [2]:
db = tlnb.db  # get the TimelinkDatabase object from the notebook
print(db.table_names())

['acts', 'aregisters', 'attributes', 'class_attributes', 'classes', 'entities', 'geoentities', 'kleiofiles', 'links', 'objects', 'persons', 'relations', 'rentities', 'sources', 'syslog', 'syspar']


### Creating person Python objects

In [3]:
from timelink.api.models import Entity, Person

p1 = Person(id='jd', name='John Doe',sex='m', obs="A person")
p2 = Person(id='jane', name='Jane Doe', sex='f', obs="Another person")
print(p1)
print(p2)

person$John Doe/m/id=jd/obs=A person
person$Jane Doe/f/id=jane/obs=Another person


### Create a database session and add the python objects

In [4]:
with db.session() as session:
    session.add(p1)
    session.add(p2)
    session.commit()



### Query with SQLAlchemy ORM query model

Use the SQLAlchemy models to query the database.

For a full tutorial see: https://docs.sqlalchemy.org/en/20/orm/queryguide/


#### Search people using entity attributes (columns in the database)

See https://docs.sqlalchemy.org/en/20/orm/quickstart.html#simple-select

In [5]:

with db.session() as session:
    r = session.query(Person).where(Person.sex == 'f').all()
    print(r)

[Person(id=Entity(id="jane", pom_class="person",inside="None", the_order=None, the_level=None, the_line=None, groupname="None", updated=2024-10-15 10:18:07.221009, indexed=None,), name="Jane Doe", sex="f", obs=Another person)]


Co-pilot normally helps if you add a previous commnent

In [6]:
# search for people with name starting with "Jane"

In [7]:
with db.session() as session:
    r = session.query(Person).where(Person.name.like('Jane%')).all()
    print(r)

[Person(id=Entity(id="jane", pom_class="person",inside="None", the_order=None, the_level=None, the_line=None, groupname="None", updated=2024-10-15 10:18:07.221009, indexed=None,), name="Jane Doe", sex="f", obs=Another person)]


Note that you can have the result in kleio format

In [8]:
with db.session() as session:
    r = session.query(Person).where(Person.name.like('Jane%')).all()
    for p in r:
        print(p.to_kleio())

person$Jane Doe/f/id=jane/obs=Another person


### Adding attribute and relations for people objects

#### Create some attributes and relations 

In [9]:
from timelink.api.models import Attribute, Relation

with db.session() as session:
    a1 = Attribute(id='jd1', the_type='age', the_value="33", the_date="2024-10-05", obs="John's age")
    jd = session.get(Person, 'jd')
    jd.add_attribute(a1)
    a2 = Attribute(id='jd2', the_type='age', the_value="11", the_date="2024-10-05", obs="Jane's age")
    jane = session.get(Person, 'jane')
    jane.add_attribute(a2)
    r1 = Relation(id='jd_jane',
                    the_type='kinship',
                    the_value='father',
                    the_date='2024-10-05',
                    inside='jd',
                    origin='jd',
                    destination='jane',
                    obs="John is the father of Jane")
    session.add(r1)
    session.commit()


We can know retrieve the attributes

In [10]:
with db.session() as session:
    jd = session.get(Person, 'jd')
    print(jd.attributes)
    print(jd.rels_out)
    jane = session.get(Person, 'jane')
    print(jane.attributes)
    print(jane.rels_in)

[Attribute(id=Entity(id="jd1", pom_class="attribute",inside="jd", the_order=None, the_level=None, the_line=None, groupname="None", updated=2024-10-15 10:18:07.453431, indexed=None,), entity="jd", the_type="age", the_value="33", the_date="2024-10-05"", obs=John's age)]
[Relation(id=Entity(id="jd_jane", pom_class="relation",inside="jd", the_order=None, the_level=None, the_line=None, groupname="None", updated=2024-10-15 10:18:07.451651, indexed=None,), origin="jd", destination="jane", the_type="kinship", the_value="father", the_date="2024-10-05"", obs=John is the father of Jane)]
[Attribute(id=Entity(id="jd2", pom_class="attribute",inside="jane", the_order=None, the_level=None, the_line=None, groupname="None", updated=2024-10-15 10:18:07.453434, indexed=None,), entity="jane", the_type="age", the_value="11", the_date="2024-10-05"", obs=Jane's age)]
[Relation(id=Entity(id="jd_jane", pom_class="relation",inside="jd", the_order=None, the_level=None, the_line=None, groupname="None", updated=20

Relations:
- `origin` and `destination` contain the ids of the related entities
- `org` and `dest`  contain the ORM managed related entities

In [11]:
with db.session() as session:
    # get John Doe, the father
    jd = session.get(Person, 'jd')
    # get the first relation
    rel = jd.rels_out[0]
    # get the destination entity id
    print('id =', rel.destination)
    # get the destination entity (note that print will show the entity)
    print('entity =',rel.dest)

id = jane
entity = 

AttributeError: 'Relation' object has no attribute 'class'

Same from the other side (daughter)

In [12]:
with db.session() as session:
    jane = session.get(Person, 'jane')
    rel = jane.rels_in[0]
    print('id =', rel.origin)
    print('entity =\n', rel.org)


id = jd
entity =
 person$John Doe/m/id=jd/obs=A person
rel$kinship/father/Jane Doe/2024-10-05/obs=John is the father of Jane
  attribute$age/33/2024-10-05/obs=John's age


And the kleio representation will show the attributes and the relations

In [ ]:
with db.session() as session:
    jd = session.get(Person, 'jd')
    print(jd.to_kleio())
    jane = session.get(Person, 'jane')
    print(jane.to_kleio())

person$John Doe/m/id=jd/obs=A person
  rel$kinship/father/Jane Doe/2024-10-05/jane/2024-10-05/obs=John is the father of Jane
  attribute$age/33/2024-10-05/obs=John's age


person$Jane Doe/f/id=jane/obs=Another person
  rel$kinship/father/Jane Doe/2024-10-05/jane/2024-10-05/obs=John is the father of Jane
  attribute$age/11/2024-10-05/obs=Jane's age


The attributes are also in their own table

In [16]:
from sqlalchemy.sql import text

# print(db.table_names())
# print(db.describe('attributes'))
with db.session() as session:
    sql = text("SELECT the_type, the_value, the_date "
                "FROM attributes")
    result = session.execute(sql)
    for row in result:
        print(row)

('age', '33', '2024-10-05')
('age', '11', '2024-10-05')


## Searching the inherintace hierarchy

https://docs.sqlalchemy.org/en/20/orm/queryguide/inheritance.html

### Search for people with certain attributes

see: https://docs.sqlalchemy.org/en/20/orm/quickstart.html#select-with-join

In [40]:
from sqlalchemy.sql import select
from sqlalchemy.orm import aliased
from timelink.api.models import Attribute, Person

# when joining tables, we need to use aliases
# we use db.get_model to get an aliased model
# see  https://docs.sqlalchemy.org/en/20/errors.html#error-xaj2

PersonAliased = aliased(Person)  # or db.get_model('Person')

with db.session() as session:
    stmt = (  # note the parentheses
        select(PersonAliased)
        .join(PersonAliased.attributes)
        .where(Attribute.the_type == "age")
    )
    people = session.scalars(stmt).all()
    for person in people:
        print(person)



person$John Doe/m/id=jd/obs=A person
  attribute$age/35/2024-10-05/obs=John's age
person$Jane Doe/f/id=jane/obs=Another person
  attribute$age/11/2024-10-05/obs=Jane's age


In [42]:
PersonAliased = db.get_model('person')  # or aliased(Person)

with db.session() as session:
    stmt = (  # note the parentheses
        select(PersonAliased)
        .join(PersonAliased.attributes)
        .where(Attribute.the_type == 'age')
        .where(Attribute.the_value < "30")
    )
    under_thirty = session.execute(stmt).scalars().all()
    print("Under thirty:")
    for p in under_thirty:
        print(p)

Under thirty:
person$Jane Doe/f/id=jane/obs=Another person
  attribute$age/11/2024-10-05/obs=Jane's age


## Inspecting the database


### Get the known ORM models

Note that importing kleio files can add to the available ORM models.

In [43]:
db.get_models_ids()

['attribute',
 'relation',
 'act',
 'source',
 'aregister',
 'person',
 'object',
 'geoentity',
 'rentity',
 'class',
 'entity']

### Get the columns of a model

In [44]:
cols = db.get_columns('attribute')
for col in cols:
    print(col.table, col.name, col.type, col.foreign_keys)

entities id VARCHAR set()
entities class VARCHAR set()
entities inside VARCHAR {ForeignKey('entities.id')}
entities the_order INTEGER set()
entities the_level INTEGER set()
entities the_line INTEGER set()
entities groupname VARCHAR set()
entities updated DATETIME set()
entities indexed DATETIME set()
attributes id VARCHAR {ForeignKey('entities.id')}
attributes entity VARCHAR {ForeignKey('entities.id')}
attributes the_type VARCHAR set()
attributes the_value VARCHAR set()
attributes the_date VARCHAR set()
attributes obs VARCHAR set()


Also with the model class

In [45]:
from timelink.api.models import Entity, Person, Attribute, Relation

db.describe(Relation)

entities id VARCHAR 
entities class VARCHAR 
entities inside VARCHAR {ForeignKey('entities.id')}
entities the_order INTEGER 
entities the_level INTEGER 
entities the_line INTEGER 
entities groupname VARCHAR 
entities updated DATETIME 
entities indexed DATETIME 
relations id VARCHAR {ForeignKey('entities.id')}
relations origin VARCHAR {ForeignKey('entities.id')}
relations destination VARCHAR {ForeignKey('entities.id')}
relations the_type VARCHAR 
relations the_value VARCHAR 
relations the_date VARCHAR 
relations obs VARCHAR 


### Getting the tables

By using tables in queries the inheritance system is by passed
and only the columns specific to a table are used.

In [46]:
db.table_names()

['acts',
 'aregisters',
 'attributes',
 'class_attributes',
 'classes',
 'entities',
 'geoentities',
 'kleiofiles',
 'links',
 'objects',
 'persons',
 'relations',
 'rentities',
 'sources',
 'syslog',
 'syspar']

### Check columns of a table

In [47]:
db.describe('attributes')

attributes id VARCHAR {ForeignKey('entities.id')}
attributes entity VARCHAR {ForeignKey('entities.id')}
attributes the_type VARCHAR 
attributes the_value VARCHAR 
attributes the_date VARCHAR 
attributes obs VARCHAR 


## Queries in SQL, use ORM to fetch full entities

Using tables and columns directly more efficient queries
can be done. Use ids from results to fetch the full entity.

In [12]:
from sqlalchemy import select, text
from timelink.api.models import Entity
sql = text("SELECT p.id "
           "FROM persons p, attributes a "
           "WHERE p.id = a.entity "
           "AND a.the_type = 'age' "
           "AND the_value < '30'")

with db.session() as session:
    result = session.execute(sql).scalars().all()
    for id in result:
        person = session.get(Entity, id)
        # note that the correct type of entity is returned
        print(person.to_kleio())



AttributeError: 'Relation' object has no attribute 'class'

## Changing data

#### Changing the value of an attribute

In [55]:
from timelink.api.models import Person

with db.session() as session:
    p1 = session.get(Person, 'jd')
    print(p1)
    # get the age attribute
    a1 = [atr for atr in p1.attributes if atr.the_type == 'age'][0]
    print("before:", a1)

    a1.the_value = str(int(a1.the_value) + 5)  #

    print("after:", a1)
    session.commit()
    print("New version of entity")
    print(p1.to_kleio())

person$John Doe/m/id=jd/obs=A person
  attribute$age/45/2024-10-05/obs=John's age
before: attribute$age/45/2024-10-05/obs=John's age
after: attribute$age/50/2024-10-05/obs=John's age
New version of entity
person$John Doe/m/id=jd/obs=A person
  attribute$age/50/2024-10-05/obs=John's age


Confirm persistence of changes data

In [57]:
with db.session() as session:
    p1 = session.get(Person, 'jd')
    # get the age attribute
    a1 = [atr for atr in p1.attributes if atr.the_type == 'age'][0]
    print("after:", a1)
    print(p1)

after: attribute$age/50/2024-10-05/obs=John's age
person$John Doe/m/id=jd/obs=A person
  attribute$age/50/2024-10-05/obs=John's age


## Deleting data